In [1]:
from llama_index.core import Document
from llama_index.core.node_parser import JSONNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding


In [2]:
# configure

filename = "recipes_flat.json"


In [3]:
with open(f"data/loader/{filename}", "r", encoding="utf-8") as file:
    document = Document(
        text=file.read(),
        metadata={"filename": filename},
    )
print(len(document.text))


2091083


In [4]:
splitter = JSONNodeParser()
nodes = splitter.get_nodes_from_documents([document], show_progress=False)
print(len(nodes))


1397


In [5]:
for ix, node in enumerate(nodes[0:5]):
    print(f">>>{ix} {node.id_}")
    print("Metadata", node.metadata)
    print("Text", node.text[:200])
    print("\n\n")


>>>0 2fc3af99-3b17-4f02-8a56-126d7b2bfa26
Metadata {'filename': 'recipes_flat.json'}
Text category Bebidas
name Margarita Cristalina
source https://www.kiwilimon.com/receta/bebidas/cocteles/margarita-cristalina
prep_time 5 mins
cook_time N/A
difficulty Baja
ingredients 1 1/2 onzas de Tequi



>>>1 5116a0e4-672e-4238-b083-53afc61e8466
Metadata {'filename': 'recipes_flat.json'}
Text category Bebidas
name Gin Con Lichi
source https://www.kiwilimon.com/receta/bebidas/cocteles/gin-con-lichi
prep_time 5 mins
cook_time N/A
difficulty Baja
ingredients suficiente de hielo, en cubos
1 1/



>>>2 39ee8311-06cf-4a26-b7b6-c7abde4d3707
Metadata {'filename': 'recipes_flat.json'}
Text category Bebidas
name Martini 007
source https://www.kiwilimon.com/receta/bebidas/cocteles/martini-007
prep_time 5 mins
cook_time N/A
difficulty Baja
ingredients 2 onzas de ginebra
1 onza de vodka, en



>>>3 85da6084-57fd-4155-8447-fb9bdd040da8
Metadata {'filename': 'recipes_flat.json'}
Text category Bebidas
name Agua D

In [6]:
import re

import chromadb

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.openai import OpenAI

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

## import settings from llama_index
from llama_index.core import Settings
from chromadb.config import Settings as StChroma


In [23]:
# configure the llm and the embedding model
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
)

Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


In [8]:
chroma_client = chromadb.PersistentClient(
    path="./chroma_recipe", settings=StChroma(allow_reset=True)
)
chroma_collection = chroma_client.get_or_create_collection(name="recipes")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes, storage_context=storage_context)


In [9]:
vector_store_info = VectorStoreInfo(
    content_info="Recipes from the web to eat in diferent ocasions",
    metadata_info=[
        MetadataInfo(
            name="filename",
            type="str",
            description=(
                "The filename of the document where the recipe was extracted from"
            ),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index, vector_store_info=vector_store_info, similarity_top_k=2
)


In [10]:
retriever.retrieve("Give me a recipe with chocolate")


[NodeWithScore(node=TextNode(id_='093211e3-34d1-409f-b693-2e9b518ec601', embedding=None, metadata={'filename': 'recipes_flat.json'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3137aa99-4b50-4d02-a7da-0fb91e8e80ab', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'recipes_flat.json'}, hash='f97ac26fbb96bc6da10d33567380c98af7e8ec0589da4ace2435cd5fa39c259e'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='350d9405-cb0d-4119-83dd-02788f6fe3cb', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': 'recipes_flat.json'}, hash='dca514c135bd149d0d07a540847ad6eebd83cf861e9fca795169dbb86c3b2c5e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='64bfef35-5e53-4459-bef2-07b56800404c', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='aba24d4945562b0073af7910ff5ee7c9ddc0dea104bfcd9616737293bf1e9009')}, text='category Postres\nname Crema Batida Sabor Chocolate\nsource http

In [24]:
engine = index.as_chat_engine(chat_mode="best", llm=llm, verbose=True)
result = engine.query("Dame una receta sencilla de preparar")


Added user message to memory: Dame una receta sencilla de preparar
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"receta sencilla de preparar"}
Got output: Una receta sencilla de preparar es la Lasana Fácil en Microondas. Tiene un tiempo de preparación de 35 minutos y es de dificultad baja. Los ingredientes incluyen mantequilla, harina, leche, mayonesa, queso manchego, carne molida de res, salsa de tomate y pasta para lasagna. La preparación consiste en hacer una salsa, cocinar la carne, armar la lasagna en un bote y calentarla en el microondas. Es ideal para disfrutar de una comida deliciosa sin complicaciones.



In [27]:
result = engine.query("Dame una bebida con limón")


Added user message to memory: Dame una bebida con limón
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"receta de bebida con limón"}
Got output: Una receta de bebida con limón es la Limonada De Pepino Con Ron. Para prepararla, necesitas los siguientes ingredientes: suficiente de hielo, 2 onzas de Bacardí® Limón Verde, 1 onza de concentrado de pepino, 1/2 onzas de jarabe natural, 1/2 onzas de jugo de limón, 100 mililitros de agua mineral (o hasta llenar el vaso) y hierbabuena al gusto para decorar. 

La preparación consiste en servir hielo en un vaso, verter el Bacardí® Limón Verde, el concentrado de pepino, el jarabe natural y el jugo de limón. Mezcla con una bailarina y agrega el agua mineral. Decora con hierbabuena y disfruta. 

Otra opción es el Limonic, que lleva también 2 onzas de Bacardí® Limón Verde, 200 mililitros de agua tónica, suficiente de hielo y rodajas de limón para decorar. Para su preparación, sirve hielo en un vaso, vierte el Bacardí®

In [28]:
print(result)


Aquí tienes dos recetas de bebidas con limón:

1. **Limonada De Pepino Con Ron**:
   - **Ingredientes**:
     - Hielo
     - 2 onzas de Bacardí® Limón Verde
     - 1 onza de concentrado de pepino
     - 1/2 onza de jarabe natural
     - 1/2 onza de jugo de limón
     - 100 mililitros de agua mineral (o hasta llenar el vaso)
     - Hierbabuena al gusto para decorar

   - **Preparación**:
     1. Sirve hielo en un vaso.
     2. Vierte el Bacardí® Limón Verde, el concentrado de pepino, el jarabe natural y el jugo de limón.
     3. Mezcla con una cuchara.
     4. Agrega el agua mineral.
     5. Decora con hierbabuena y disfruta.

2. **Limonic**:
   - **Ingredientes**:
     - Hielo
     - 2 onzas de Bacardí® Limón Verde
     - 200 mililitros de agua tónica
     - Rodajas de limón para decorar

   - **Preparación**:
     1. Sirve hielo en un vaso.
     2. Vierte el Bacardí® Limón Verde y el agua tónica.
     3. Mezcla y decora con una rodaja de limón.
     4. ¡Disfruta!

Ambas son refrescant

In [29]:
engine.get


{}

## Vector Retriever (with chroma in local)


In [32]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI


In [42]:
chroma_client = chromadb.PersistentClient(
    path="./chroma_recipe", settings=StChroma(allow_reset=True)
)
chroma_collection = chroma_client.get_or_create_collection(name="recipes")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)


In [43]:
vector_store_info = VectorStoreInfo(
    content_info="Recipes from the web to eat in diferent ocasions",
    metadata_info=[
        MetadataInfo(
            name="filename",
            type="str",
            description=(
                "The filename of the document where the recipe was extracted from"
            ),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index,
    vector_store_info=vector_store_info,
    similarity_top_k=3,
    verbose=True,
)


In [47]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
)

query_engine = RetrieverQueryEngine.from_args(
    retriever,
    llm=llm,
)


In [ ]:
query_engine


In [49]:
resp = query_engine.query("Recomiendame algo caliente para el frío")


Using query str: Recetas calientes para el frío
Using filters: []


In [50]:
print(resp)


Te recomiendo preparar un delicioso caldo de queso. Es una opción reconfortante y fácil de hacer. Solo necesitas ingredientes como cebolla, jitomate, papas, chiles, leche y queso fresco. Se cocina todo junto para obtener un caldo caliente y sabroso, perfecto para disfrutar en un día frío. ¡Sirve bien caliente y disfruta!
